In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [3]:
from sensorcnn.visualization.examples import *
from sensorcnn.dataset.examples import *

### Encode labels

In [4]:
label_to_idx = {l:i for i, l in enumerate(labels)}

def encode(label, n_labels=6):
    """
    Returns a binary vector with all zeros or one at the label position
    
    example:
    -------
    ecnode(4, n_labels=6)
    = np.array(0, 0, 0, 1, 0, 0)
    
    encode(None, n_labels=6)
    = np.array(0, 0, 0, 0, 0, 0)
    """
    y = np.zeros(n_labels)
    if label is not None and 0 <= label < n_labels:
        y[label] = 1
        
    return y

### Label the first 50 samples as setup

In [5]:
def get_setup_labels(example, setup_threshold=50, n_labels=6):
    """Pick the first 50 samples and label them as setup"""
    ys = []
    i = 0
    for l in examples['label']:
        label_index = label_to_idx.get(labels_mapping.get(l))
        if label_index is None:
            i = 0
            ys.append(np.zeros(n_labels))
        else:
            i += 0
            if i > setup_threshold:
                ys.append(np.zeros(n_labels))
            else:
                ys.append(encode(label_index))
    return np.array(ys)

In [6]:
root_directory = "/Users/mostafa/Google Drive/Exercise Data/"
n_features = 3
n_labels = 6
X = np.zeros((0, 3))
Y = np.zeros((0, 6))
for filename in csv_file_iterator(root_directory):
    examples = load_from_csv(filename)
    n_examples = len(examples)
    X = np.append(X, get_features(examples), axis=0)
    ys = get_setup_labels(examples)
    Y = np.append(Y, ys.reshape((n_examples, n_labels)), axis=0)

/Users/mostafa/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [7]:
print(X.shape)
print(Y.shape)

(6557118, 3)
(6557118, 6)


In [12]:
np.savez_compressed("/Users/mostafa/Desktop/muvr-6-exercises.npz", X, Y)

### Get exercise setup data only and drop the other samples

In [88]:
def get_labeled(X, Y):
    """Return only examples with setup labels and drop other examples"""
    assert X.shape[0] == Y.shape[0], "X and Y lengthes don't match"
    labeled_xs = []
    labeled_ys = []
    for x, y in zip(X, Y):
        if sum(y) == 0:
            continue

        labeled_xs.append(x)
        labeled_ys.append(y)

    return np.array(labeled_xs), np.array(labeled_ys)

In [89]:
X_labeled, Y_labeled = get_labeled(X, Y)

###  Examples augmentation - put each 50 sample into 1 50*3 sample

In [76]:
def augment_examples(X, Y, new_sample_size=50, label_merge=np.average):
    """"""
    assert X.shape[0] == Y.shape[0], "X and Y lengthes don't match"
    n_samples = X.shape[0]
    n_features = X.shape[1]
    n_labels = Y.shape[1]
    end = n_samples - (n_samples % new_sample_size)
    new_X = X[:end, :].reshape((int(end/new_sample_size), int(n_features*new_sample_size)))
    ys = []
    for y in Y[:end, :].reshape((int(end/new_sample_size), int(n_labels*new_sample_size))):
        ys.append(label_merge(y.reshape((new_sample_size, n_labels)), axis=0))

    new_Y = np.array(ys)
    return new_X, new_Y

In [77]:
new_X, new_Y = augment_examples(X_labeled, Y_labeled)

In [78]:
print(new_X.shape)
print(new_Y.shape)

(18530, 150)
(18530, 6)


In [79]:
np.savez_compressed("/Users/mostafa/Desktop/muvr-6-labeledonly-exercises.npz", new_X, new_Y)

### Train an MLP model

In [91]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(new_X, new_Y, test_size=0.33, random_state=42)

In [95]:
from sensorcnn.train.mlp_model import *
#TODO